This the code for walking through the lesson examples for your reference.

## Code for Building Synthetic Dataset

In [ ]:
# # Explanation of Code Block: Importing Libraries for Data Manipulation
#
# This block uses the `pandas` and `numpy` libraries, which are essential for data analysis and manipulation in Python.
#
# - `import pandas as pd`: Imports the pandas library and assigns it the alias `pd`. Pandas provides powerful data structures like DataFrame for handling structured data.
# - `import numpy as np`: Imports the numpy library and assigns it the alias `np`. Numpy is used for numerical operations and working with arrays.
#
import pandas as pd
import numpy as np


In [12]:
# Explanation of Code Block: Defining Synthetic Dataset Parameters
#
# This block sets up key parameters for generating a synthetic line-level dataset.
# - NUMBER_RECORDS: Specifies the total number of records (rows) to generate in the dataset.
# - NUMBER_ENCOUNTERS: Sets the number of unique encounter IDs to simulate different patient visits.
# - NUMBER_PATIENTS: Sets the number of unique patient IDs to represent individual patients.
# These variables are used throughout the notebook to control the size and structure of the synthetic data.

#build synthetic line level example
NUMBER_RECORDS = 100000
NUMBER_ENCOUNTERS = 7800
NUMBER_PATIENTS = 1000

In [13]:
# Explanation of Code Block: Creating Random Code Sets for Synthetic EHR Data
#
# This block uses numpy's `arange` function to generate ranges of integers, which are then converted to string identifiers for different medical code sets.
# - `dx_code_list`: Creates a list of diagnosis codes, formatted as "dx_code_X" for X in 1 to 99,999.
# - `procedure_code_list`: Creates a list of procedure codes, formatted as "procedure_code_X" for X in 0 to 72,999.
# - `medication_code_list`: Creates a list of medication codes, formatted as "medication_code_X" for X in 0 to 9,999.
# - `lab_code_list`: Creates a list of lab codes, formatted as "lab_code_X" for X in 0 to 9,999.
# These lists are used to randomly assign codes to synthetic patient records in later steps.
#

# Create random list of code sets for diagnosis, procedure, medication, and lab codes
dx_code_list = ["dx_code_" + str(x) for x in np.arange(1,100000)]
procedure_code_list =["procedure_code_" + str(x) for x in np.arange(0,73000)]
medication_code_list = ["medication_code_" + str(x) for x in np.arange(0,10000)]
lab_code_list = ["lab_code_" + str(x) for x in np.arange(0,10000)]

In [14]:
# Explanation of Code Block: Creating Patient and Encounter ID Lists
#
# This block generates unique identifiers for patients and encounters using Python list comprehensions and the numpy arange function.
# - `patient_id_list`: Creates a list of patient IDs in the format "udacity_health_patient_id_X" where X ranges from 1 to NUMBER_PATIENTS (inclusive). Uses `np.arange(1, NUMBER_PATIENTS + 1)` to generate the range.
# - `encounter_id_list`: Creates a list of encounter IDs in the format "udacity_health_encounter_id_X" where X ranges from 1 to NUMBER_ENCOUNTERS (inclusive). Uses `np.arange(1, NUMBER_ENCOUNTERS + 1)` to generate the range.
# These lists are used to assign unique identifiers to synthetic patient and encounter records in the dataset.
#

patient_id_list = ["udacity_health_patient_id_" + str(x) for x in np.arange(1, NUMBER_PATIENTS +1)]
encounter_id_list = ["udacity_health_encounter_id_" + str(x) for x in np.arange(1, NUMBER_ENCOUNTERS +1)]

In [ ]:
# Explanation of Code Block: Random Value Selection Utility Function
#
# This block defines a utility function for randomly selecting values from a provided list, using a probability distribution.
# - Library Used: numpy (np)
#   - np.random.dirichlet: Generates a probability distribution over the list elements, ensuring the probabilities sum to 1.
#     - Parameter: np.ones(len(field_value_list)) creates a uniform prior for the Dirichlet distribution.
#     - size=1: Generates one probability vector.
#   - np.random.choice: Randomly selects elements from the input list according to the generated probability distribution.
#     - Parameters:
#         - field_value_list: The list of possible values to sample from.
#         - number_records: The number of samples to draw.
#         - p=field_prob_dist: The probability distribution for sampling.
# The function returns a numpy array of randomly selected values, useful for generating synthetic data with controlled randomness.
#
def random_value_selection(field_value_list, number_records):
    #build normal probability distribution
    field_prob_dist = np.random.dirichlet(np.ones(len(field_value_list)), size=1)[0]
    #build random value list for field
    field_random_values = np.random.choice(field_value_list, number_records, p=field_prob_dist)
    return field_random_values

In [16]:
# Explanation of Code Block: Generating Random Values for Encounters
#patient_values = random_value_selection(patient_id_list, NUMBER_RECORDS)
encounter_values = random_value_selection(encounter_id_list, NUMBER_RECORDS)

In [19]:
# Explanation of Code Block: Generating Random Values for Medical Codes
procedure_values = random_value_selection(procedure_code_list, NUMBER_RECORDS)
medication_values = random_value_selection(medication_code_list, NUMBER_RECORDS)
lab_values = random_value_selection(lab_code_list, NUMBER_RECORDS)

In [20]:
# Explanation of Code Block: Assigning Procedure, Medication, and Lab Codes to Synthetic Records
#
# This block uses numpy to randomly assign one of three types of codes (procedure, medication, lab) to each synthetic record.
# - Library Used: numpy (np)
#   - np.random.choice: Randomly selects values from a list, with probabilities specified by the `p` parameter.
#     - Parameters:
#         - [0, 1, 2]: The possible choices, representing procedure, medication, and lab codes.
#         - NUMBER_RECORDS: The number of selections to make (one per record).
#         - p: Probability distribution for each choice, generated by np.random.dirichlet.
#   - np.random.dirichlet: Generates a probability distribution over three categories, ensuring the probabilities sum to 1.
#     - Parameter: np.ones(3) creates a uniform prior for the Dirichlet distribution.
#     - size=1: Generates one probability vector.
# - list(zip(...)): Combines the procedure, medication, lab values, and the random choice into tuples for each record.
# - List comprehensions: Select the appropriate code for each record based on the random choice, setting the other codes to np.nan.
#     - If triplet_prob_choice is 0, assign procedure code; if 1, assign medication code; if 2, assign lab code.
#
triplet_prob_choice = np.random.choice([0, 1, 2], NUMBER_RECORDS, p= np.random.dirichlet(np.ones(3), size=1)[0] )
line_triplet_values = list(zip(procedure_values, medication_values, lab_values, triplet_prob_choice))
selected_procedure_values = [x[0] if x[3] == 0 else np.nan for x in line_triplet_values ]
selected_medication_values = [x[1] if x[3] == 1 else np.nan for x in line_triplet_values]
selected_lab_values = [x[2] if x[3] == 2 else np.nan for x in line_triplet_values]

In [ ]:
# Explanation of Code Block: Creating a DataFrame for Synthetic EHR Data
#add label
patient_label_mapping = dict(zip( patient_id_list, np.random.choice([0, 1], NUMBER_PATIENTS, replace=True,
                                                                    p=[0.88, 0.12]) ))
label_values = [patient_label_mapping[x] for x in patient_values]

In [22]:
# This block creates a pandas DataFrame to hold the synthetic electronic health record (EHR) data.
line_df = pd.DataFrame({ "ENCOUNTER_ID": encounter_values,
                        "PATIENT_ID": patient_values,
                        "PRINCIPAL_DIAGNOSIS_CODE": dx_values,
                        "PROCEDURE_CODE": selected_procedure_values,
                        "MEDICATION_CODE": selected_medication_values,
                        "LAB_CODE": selected_lab_values,
                        "LABEL": label_values
                       })

In [23]:
# Explanation of Code Block: Saving the Synthetic EHR Data to a CSV File
#line_df.to_csv("./data/SYNTHETIC_EHR_DATASET.csv", index=False)

## 1. Converting Line to Encounter Representation

### Load Synthetic EHR Line Dataset

In [24]:
# This block reads the synthetic EHR dataset from a CSV file into a pandas DataFrame.
ehr_line_df = pd.read_csv("./data/SYNTHETIC_EHR_DATASET.csv")

In [25]:
# Explanation of Code Block: Displaying the First Few Rows of the DataFrame
ehr_line_df.head()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1933,udacity_health_patient_id_817,dx_code_56522,procedure_code_20005,NaN,NaN,0
1,udacity_health_encounter_id_5664,udacity_health_patient_id_594,dx_code_39264,NaN,medication_code_7471,NaN,0
2,udacity_health_encounter_id_1946,udacity_health_patient_id_169,dx_code_83619,NaN,NaN,lab_code_5311,0
3,udacity_health_encounter_id_1528,udacity_health_patient_id_870,dx_code_97434,procedure_code_31111,NaN,NaN,0
4,udacity_health_encounter_id_7185,udacity_health_patient_id_722,dx_code_68924,NaN,medication_code_2250,NaN,0


In [26]:
# Explanation of Code Block: Filtering the DataFrame for a Specific Encounter ID
ehr_line_df[ehr_line_df['ENCOUNTER_ID']=='udacity_health_encounter_id_100']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
14286,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_4198,0
19091,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_7982,NaN,0
29530,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_6603,0
34583,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_332,0
62325,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_2452,NaN,0
64967,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,NaN,lab_code_1431,0
94636,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,NaN,medication_code_279,NaN,0


In [ ]:
#note that this is for illustrative purposes only and for practicing key skills,
# the actual data representation and combinations of codes not indicative of real thing

### Convert Line to Encounter Representation

In [ ]:
# Explanation of Code Block: Defining Grouping and Non-Grouping Fields
# grouping fields
grouping_field_list = ['ENCOUNTER_ID', 'PATIENT_ID', 'PRINCIPAL_DIAGNOSIS_CODE']
non_grouped_field_list = [c for c in ehr_line_df.columns if c not in grouping_field_list]

In [ ]:
# Explanation of Code Block: Aggregating Data by Grouping Fields
#
# This block uses the pandas library to aggregate the synthetic EHR line-level data into an encounter-level representation.
# - Library Used: pandas (pd)
#   - `groupby`: Groups the DataFrame by the specified fields in `grouping_field_list` (e.g., 'ENCOUNTER_ID', 'PATIENT_ID', 'PRINCIPAL_DIAGNOSIS_CODE').
#   - `agg`: Aggregates the non-grouped fields (e.g., 'PROCEDURE_CODE', 'MEDICATION_CODE', 'LAB_CODE', 'LABEL') using a custom lambda function.
#     - The lambda function collects all non-NaN values for each group into a list, preserving all codes and labels associated with each encounter.
#   - `reset_index`: Resets the index of the resulting DataFrame to provide a clean, consecutive integer index.
# - Key Parameters:
#   - `grouping_field_list`: List of columns to group by.
#   - `non_grouped_field_list`: List of columns to aggregate.
#   - `lambda x: list([y for y in x if y is not np.nan])`: Aggregation function to collect non-NaN values into lists.
#
encounter_df = ehr_line_df.groupby(grouping_field_list)[non_grouped_field_list].agg(lambda x:
                                                        list([y for y in x if y is not np.nan ] ) ).reset_index()

In [30]:
# Explanation of Code Block: Displaying the First Few Rows of the Encounter DataFrame
encounter_df[0:5]

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,udacity_health_encounter_id_10,udacity_health_patient_id_188,dx_code_74047,[],"[medication_code_7789, medication_code_3560, m...",[],"[1, 1, 1, 1]"
2,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,[],"[medication_code_7982, medication_code_2452, m...","[lab_code_4198, lab_code_6603, lab_code_332, l...","[0, 0, 0, 0, 0, 0, 0]"
3,udacity_health_encounter_id_1000,udacity_health_patient_id_525,dx_code_61569,[],[medication_code_4036],[],[0]
4,udacity_health_encounter_id_1001,udacity_health_patient_id_950,dx_code_90172,[procedure_code_30555],"[medication_code_6755, medication_code_5045]",[lab_code_9112],"[0, 0, 0, 0]"


In [31]:
## Explanation of Code Block: Filtering the Encounter DataFrame for a Specific Encounter ID
ehr_line_df[ehr_line_df['ENCOUNTER_ID']=='udacity_health_encounter_id_1']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
3246,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_2350,NaN,0
7901,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_8630,NaN,0
11765,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,NaN,lab_code_8835,0
11950,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,procedure_code_58552,NaN,NaN,0
16057,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_4030,NaN,0
17961,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,procedure_code_39776,NaN,NaN,0
24877,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_431,NaN,0
26235,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_7435,NaN,0
35490,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,NaN,lab_code_9859,0
42855,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,NaN,medication_code_4338,NaN,0


In [32]:
# Explanation of Code Block: Filtering the Encounter DataFrame for a Specific Encounter ID
encounter_df[encounter_df['ENCOUNTER_ID']=='udacity_health_encounter_id_1']

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 2. Converting Encounter to Longitudinal Representation

In [33]:
## Explanation of Code Block: Displaying the First Few Rows of the Encounter DataFrame
encounter_df.head()

,ENCOUNTER_ID,PATIENT_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_encounter_id_1,udacity_health_patient_id_186,dx_code_15406,"[procedure_code_58552, procedure_code_39776, p...","[medication_code_2350, medication_code_8630, m...","[lab_code_8835, lab_code_9859, lab_code_9032, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,udacity_health_encounter_id_10,udacity_health_patient_id_188,dx_code_74047,[],"[medication_code_7789, medication_code_3560, m...",[],"[1, 1, 1, 1]"
2,udacity_health_encounter_id_100,udacity_health_patient_id_585,dx_code_71465,[],"[medication_code_7982, medication_code_2452, m...","[lab_code_4198, lab_code_6603, lab_code_332, l...","[0, 0, 0, 0, 0, 0, 0]"
3,udacity_health_encounter_id_1000,udacity_health_patient_id_525,dx_code_61569,[],[medication_code_4036],[],[0]
4,udacity_health_encounter_id_1001,udacity_health_patient_id_950,dx_code_90172,[procedure_code_30555],"[medication_code_6755, medication_code_5045]",[lab_code_9112],"[0, 0, 0, 0]"


In [34]:
## Explanation of Code Block: Defining Patient Grouping and Non-Patient Aggregation Fields
patient_grouping_field_list = ["PATIENT_ID"]
non_patient_agg_field_list = [c for c in encounter_df.columns if c not in patient_grouping_field_list]

In [ ]:
# Explanation of Code Block: Aggregating Encounter Data by Patient ID
long_df = encounter_df.groupby(patient_grouping_field_list)[non_patient_agg_field_list].agg(lambda x:
                                                        list([y for y in x if y is not np.nan ] ) ).reset_index()

In [36]:
## Explanation of Code Block: Displaying the First Few Rows of the Long DataFrame
long_df.head()

,PATIENT_ID,ENCOUNTER_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
0,udacity_health_patient_id_1,"[udacity_health_encounter_id_1038, udacity_hea...","[dx_code_36196, dx_code_63471, dx_code_29114, ...","[[procedure_code_36285, procedure_code_21124, ...","[[medication_code_3772, medication_code_9214, ...","[[lab_code_3982, lab_code_306], [], [lab_code_...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,udacity_health_patient_id_10,"[udacity_health_encounter_id_1110, udacity_hea...","[dx_code_29609, dx_code_268, dx_code_26932, dx...","[[procedure_code_9379], [procedure_code_3052, ...","[[medication_code_7371, medication_code_2104, ...","[[lab_code_6457], [lab_code_2180, lab_code_693...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ..."
2,udacity_health_patient_id_100,"[udacity_health_encounter_id_1205, udacity_hea...","[dx_code_32095, dx_code_45376, dx_code_48998, ...","[[procedure_code_71055, procedure_code_29744, ...","[[medication_code_2399, medication_code_966, m...","[[lab_code_4928, lab_code_6524, lab_code_1713,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,udacity_health_patient_id_1000,"[udacity_health_encounter_id_1105, udacity_hea...","[dx_code_53764, dx_code_50924, dx_code_80218, ...","[[], [procedure_code_7870, procedure_code_2169...","[[medication_code_6580, medication_code_3007],...","[[], [lab_code_5468], [lab_code_7607], [lab_co...","[[1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1..."
4,udacity_health_patient_id_101,"[udacity_health_encounter_id_2058, udacity_hea...","[dx_code_13590, dx_code_29551]","[[procedure_code_45319, procedure_code_67294],...","[[medication_code_2532], [medication_code_4311...","[[lab_code_1206, lab_code_9967], []]","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]"


In [37]:
# Explanation of Code Block: Filtering the Long DataFrame for a Specific Patient ID
example_patient_history = long_df[long_df['PATIENT_ID']=='udacity_health_patient_id_310']

In [38]:
# Explanation of Code Block: Displaying the Example Patient History
example_patient_history

,PATIENT_ID,ENCOUNTER_ID,PRINCIPAL_DIAGNOSIS_CODE,PROCEDURE_CODE,MEDICATION_CODE,LAB_CODE,LABEL
211,udacity_health_patient_id_310,"[udacity_health_encounter_id_4647, udacity_hea...","[dx_code_74153, dx_code_95836, dx_code_39465, ...","[[procedure_code_40521, procedure_code_52188, ...","[[medication_code_7251, medication_code_2765, ...","[[lab_code_1372], [lab_code_1794], [lab_code_1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [39]:
# Explanation of Code Block: Extracting Encounter IDs from Example Patient History
list(example_patient_history['ENCOUNTER_ID'].values)

[['udacity_health_encounter_id_4647',
  'udacity_health_encounter_id_5010',
  'udacity_health_encounter_id_551',
  'udacity_health_encounter_id_7210',
  'udacity_health_encounter_id_7331']]

In [40]:
#    Explanation of Code Block: Extracting Principal Diagnosis Codes from Example Patient History
list(example_patient_history['PRINCIPAL_DIAGNOSIS_CODE'].values)

[['dx_code_74153',
  'dx_code_95836',
  'dx_code_39465',
  'dx_code_66358',
  'dx_code_99743']]

In [41]:
# Explanation of Code Block: Extracting Procedure Codes from Example Patient History
list(example_patient_history['PROCEDURE_CODE'].values)

[[['procedure_code_40521',
   'procedure_code_52188',
   'procedure_code_57020',
   'procedure_code_11784',
   'procedure_code_50478'],
  ['procedure_code_12696',
   'procedure_code_2873',
   'procedure_code_28392',
   'procedure_code_1398'],
  ['procedure_code_49962'],
  [],
  ['procedure_code_31171']]]

## 3. How to Split Dataset at Patient Level

#### ***Objective:*** 
- Split dataset at patient level into train and test partitions
- Validate that split was done correctly

#### Dataset Splitting Tests
- Patient data in only one partition
- Total unique number of patients across all partitions = total number unique patients in original full dataset
- Total number of rows original dataset = sum of rows across splits

In [42]:
#    Explanation of Code Block: Extracting Medication Codes from Example Patient History
PATIENT_ID_FIELD = 'PATIENT_ID'
TEST_PERCENTAGE = 0.2

In [43]:
# Explanation of Code Block: Splitting Dataset at Patient Level
def split_dataset_patient_level(df, key, test_percentage=0.2):
    df = df.iloc[np.random.permutation(len(df))]
    unique_values = df[key].unique()
    total_values = len(unique_values)
    sample_size = round(total_values * (1 - test_percentage ))
    train = df[df[key].isin(unique_values[:sample_size])].reset_index(drop=True)
    test = df[df[key].isin(unique_values[sample_size:])].reset_index(drop=True)
    return train, test

In [44]:
# Explanation of Code Block: Splitting the Encounter DataFrame into Training and Testing Sets
train_df, test_df = split_dataset_patient_level(encounter_df, PATIENT_ID_FIELD, TEST_PERCENTAGE)

In [45]:
# Explanation of Code Block: Displaying the Number of Records in Training and Testing Sets
assert len(set(train_df[PATIENT_ID_FIELD].unique()).intersection(set(test_df[PATIENT_ID_FIELD].unique()))) == 0
print("Test passed for patient data in only one partition")

Test passed for patient data in only one partition


In [46]:
# Explanation of Code Block: Asserting the Number of Unique Patients in Training and Testing Sets
assert (train_df[PATIENT_ID_FIELD].nunique()  + test_df[PATIENT_ID_FIELD].nunique()) == encounter_df[PATIENT_ID_FIELD].nunique()
print("Test passed for number of unique patients being equal!")

Test passed for number of unique patients being equal!


In [47]:
# Explanation of Code Block: Asserting the Total Number of Rows in Training and Testing Sets
assert len(train_df)  + len(test_df) == len(encounter_df)
print("Test passed for number of total rows equal!")

Test passed for number of total rows equal!


## 4. ETL with TF Dataset API and Pandas

NOTE: In some cases you may need to preprocess Pandas Dataframe to removed mixed types. In particular, remove null values and impute or remove rows (we will later impute with zero for numerical features).

In [48]:
# Explanation of Code Block: Importing TensorFlow Library
import tensorflow as tf

In [49]:
# Explanation of Code Block: Importing TensorFlow Datasets Library
swiss_dataset_path = "./data/processed_swiss.csv"
swiss_df = pd.read_csv(swiss_dataset_path)
selected_col_list = ['age', 'thalach', 'cp', 'num_label']
subset_swiss_df = swiss_df[selected_col_list]

In [50]:
# Explanation of Code Block: Displaying the First Few Rows of the Swiss Dataset
swiss_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num_label
0,32,1,1,95,0,?,0,127,0,.7,1,?,?,1
1,34,1,4,115,0,?,?,154,0,.2,1,?,?,1
2,35,1,4,?,0,?,0,130,1,?,?,?,7,3
3,36,1,4,110,0,?,0,125,1,1,2,?,6,1
4,38,0,4,105,0,?,0,166,0,2.8,1,?,?,2


In [51]:
# Explanation of Code Block: Displaying the First Few Rows of the Subset Swiss DataFrame

subset_swiss_df.head()

,age,thalach,cp,num_label
0,32,127,1,1
1,34,154,4,1
2,35,130,4,3
3,36,125,4,1
4,38,166,4,2


In [52]:
# Explanation of Code Block: Converting DataFrame to TensorFlow Dataset
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [53]:
# Explanation of Code Block: Defining Batch Size and Predictor Field for TensorFlow Dataset
BATCH_SIZE = 64
PREDICTOR_FIELD = 'num_label'
sample_tf_ds = df_to_dataset(subset_swiss_df, PREDICTOR_FIELD, batch_size=BATCH_SIZE)

In [54]:
# Explanation of Code Block: Displaying a Sample Batch from the TensorFlow Dataset
sample_feature_batch = next(iter(sample_tf_ds))[0]
sample_feature_batch

{'age': <tf.Tensor: shape=(64,), dtype=int64, numpy=
 array([56, 55, 63, 52, 67, 59, 53, 51, 61, 53, 55, 51, 55, 56, 38, 72, 51,
        52, 38, 62, 70, 52, 43, 54, 43, 56, 58, 56, 35, 69, 61, 42, 61, 62,
        42, 50, 62, 61, 57, 61, 60, 53, 57, 41, 63, 53, 54, 47, 61, 38, 62,
        60, 57, 54, 57, 47, 61, 53, 70, 64, 38, 66, 50, 64])>,
 'thalach': <tf.Tensor: shape=(64,), dtype=string, numpy=
 array([b'99', b'83', b'149', b'82', b'125', b'175', b'120', b'126',
        b'117', b'135', b'92', b'92', b'150', b'98', b'120', b'114',
        b'163', b'120', b'128', b'134', b'92', b'128', b'140', b'150',
        b'122', b'148', b'105', b'103', b'130', b'?', b'145', b'99',
        b'105', b'128', b'128', b'139', b'123', b'105', b'148', b'115',
        b'99', b'120', b'131', b'176', b'86', b'115', b'110', b'120',
        b'113', b'179', b'72', b'63', b'100', b'155', b'98', b'118', b'77',
        b'141', b'157', b'140', b'156', b'90', b'120', b'114'],
       dtype=object)>,
 'cp': <tf.Tens

In [55]:
# Explanation of Code Block: Displaying Labels from the Sample Batch of the TensorFlow Dataset
sample_label_batch = next(iter(sample_tf_ds))[1]
sample_label_batch

<tf.Tensor: shape=(64,), dtype=int64, numpy=
array([3, 1, 3, 3, 3, 1, 1, 2, 1, 2, 1, 3, 1, 1, 0, 3, 1, 1, 3, 3, 2, 2,
       0, 3, 2, 2, 1, 1, 3, 2, 2, 1, 1, 1, 1, 3, 1, 3, 4, 2, 1, 1, 3, 0,
       0, 2, 1, 3, 1, 2, 2, 1, 0, 0, 2, 1, 2, 1, 2, 1, 1, 1, 3, 4])>

## 5. Building Numerical Feature with TF Feature Column API

In [56]:
# Explanation of Code Block: Displaying the First Few Rows of the Subset Swiss DataFrame

subset_swiss_df.head()

,age,thalach,cp,num_label
0,32,127,1,1
1,34,154,4,1
2,35,130,4,3
3,36,125,4,1
4,38,166,4,2


In [57]:
# Explanation of Code Block: Calculating Mean and Standard Deviation of Age in the Subset Swiss DataFrame
age_mean = subset_swiss_df['age'].describe()['mean']
age_std = subset_swiss_df['age'].describe()['std']
print("Mean age:{}\nStandard Deviation Age:{}".format(age_mean, age_std))

Mean age:55.31707317073171
Standard Deviation Age:9.032107639562039


In [58]:
# Explanation of Code Block: Defining a Function to Normalize Numeric Features with Z-Score
import functools
def normalize_numeric_with_zscore(col, mean, std):
    return (col - mean)/std

def create_tf_numeric_feature(col, MEAN, STD,   default_value=0):
    normalizer = functools.partial(normalize_numeric_with_zscore, mean=MEAN, std=STD)
    return tf.feature_column.numeric_column(
    key=col, default_value = default_value, normalizer_fn=normalizer, dtype=tf.float64)

In [59]:
# Explanation of Code Block: Creating a TensorFlow Numeric Feature for Age
age_tf_feature = create_tf_numeric_feature('age', age_mean, age_std)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [60]:
# Explanation of Code Block: Displaying the Created TensorFlow Numeric Feature for Age
def demo(feature_column, example_batch):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch))

In [70]:
print("Example continuous field:\n{}\n".format(age_tf_feature))
print("Type of sample_feature_batch:", type(sample_feature_batch))
print("Keys in sample_feature_batch:", sample_feature_batch.keys())
print("Type of sample_feature_batch['age']:", type(sample_feature_batch['age']))
# Convert to tensor if needed
import tensorflow as tf
if not isinstance(sample_feature_batch['age'], tf.Tensor):
    sample_feature_batch['age'] = tf.convert_to_tensor(sample_feature_batch['age'])
print("Shape of sample_feature_batch['age']:", sample_feature_batch['age'].shape)
demo(age_tf_feature, sample_feature_batch)

Example continuous field:
NumericColumn(key='age', shape=(1,), default_value=(0,), dtype=tf.float64, normalizer_fn=functools.partial(<function normalize_numeric_with_zscore at 0x33469aca0>, mean=np.float64(55.31707317073171), std=np.float64(9.032107639562039)))

Type of sample_feature_batch: <class 'dict'>
Keys in sample_feature_batch: dict_keys(['age', 'thalach', 'cp'])
Type of sample_feature_batch['age']: <class 'tensorflow.python.framework.ops.EagerTensor'>
Shape of sample_feature_batch['age']: (64,)


Example continuous field:
NumericColumn(key='age', shape=(1,), default_value=(0,), dtype=tf.float64, normalizer_fn=functools.partial(<function normalize_numeric_with_zscore at 0x33469aca0>, mean=np.float64(55.31707317073171), std=np.float64(9.032107639562039)))

Type of sample_feature_batch: <class 'dict'>
Keys in sample_feature_batch: dict_keys(['age', 'thalach', 'cp'])
Type of sample_feature_batch['age']: <class 'tensorflow.python.framework.ops.EagerTensor'>
Shape of sample_feature_batch['age']: (64,)


AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

## 6. Building Categorical Features with TF Feature Column API

In [ ]:
# Explanation of Code Block: Creating a DataFrame for Categorical Example
categorical_example_df = encounter_df[['ENCOUNTER_ID', 'PRINCIPAL_DIAGNOSIS_CODE', 'LABEL']]

In [72]:
# Explanation of Code Block: Displaying the First Few Rows of the Categorical Example DataFrame
# for this task need to convert label from array to scalar value
categorical_example_df['LABEL'] = categorical_example_df['LABEL'].apply(lambda x: np.unique(x)[0])

/var/folders/sy/nnrjh0cx1897yr6m0_fq9wmc0000gq/T/ipykernel_49341/3416705905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_example_df['LABEL'] = categorical_example_df['LABEL'].apply(lambda x: np.unique(x)[0])


In [73]:
# Explanation of Code Block: Displaying the First Few Rows of the Categorical Example DataFrame
categorical_example_df.head()

,ENCOUNTER_ID,PRINCIPAL_DIAGNOSIS_CODE,LABEL
0,udacity_health_encounter_id_1,dx_code_15406,0
1,udacity_health_encounter_id_10,dx_code_74047,1
2,udacity_health_encounter_id_100,dx_code_71465,0
3,udacity_health_encounter_id_1000,dx_code_61569,0
4,udacity_health_encounter_id_1001,dx_code_90172,0


### High Cardinality for Principal Diagnosis Code

In [74]:
# Explanation of Code Block: Counting Unique Principal Diagnosis Codes in the Categorical Example DataFrame
categorical_example_df['PRINCIPAL_DIAGNOSIS_CODE'].nunique()

6752

### Generate Vocabulary File

In [75]:
# Explanation of Code Block: Creating a TensorFlow Categorical Feature for Principal Diagnosis Code
#make vocab dir
import os
#os.mkdir("./vocab/")

In [ ]:
# Explanation of Code Block: Function to Write Vocabulary File for Categorical Features
# build vocab for categorical features
def write_vocabulary_file(vocab_list, field_name, default_value, vocab_dir='./vocab/'):
    output_file_path = os.path.join(vocab_dir, str(field_name) + "_vocab.txt")
    # put default value in first row as TF requires
    vocab_list = np.insert(vocab_list, 0, default_value, axis=0)
    df = pd.DataFrame(vocab_list).to_csv(output_file_path, index=None, header=None)
    return output_file_path

def build_vocab_files(df, categorical_column_list, default_value='00'):
    vocab_files_list = []
    for c in categorical_column_list:
        v_file = write_vocabulary_file(df[c].unique(), c, default_value)
        vocab_files_list.append(v_file)
    return vocab_files_list

In [77]:
# Explanation of Code Block: Building Vocabulary Files for Categorical Features
categorical_field_list = ["PRINCIPAL_DIAGNOSIS_CODE"]
vocab_files_list = build_vocab_files(categorical_example_df, categorical_field_list)

### Build TF Dataset from Pandas Dataframe

In [78]:
# Explanation of Code Block: Creating a TensorFlow Categorical Feature for Principal Diagnosis Code
BATCH_SIZE = 64
PREDICTOR_FIELD = 'LABEL'
categorical_tf_ds = df_to_dataset(categorical_example_df, PREDICTOR_FIELD, batch_size=BATCH_SIZE)

### Use TF Feature Column API to read from vocab file

In [79]:
# Explanation of Code Block: Displaying a Sample Batch from the Categorical TensorFlow Dataset
vocab_files_list[0]

'./vocab/PRINCIPAL_DIAGNOSIS_CODE_vocab.txt'

In [80]:
# Explanation of Code Block: Creating a TensorFlow Categorical Feature for Principal Diagnosis Code
principal_diagnosis_vocab = tf.feature_column.categorical_column_with_vocabulary_file(
            key="PRINCIPAL_DIAGNOSIS_CODE", vocabulary_file = vocab_files_list[0], num_oov_buckets=1)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
INFO:tensorflow:vocabulary_size = 6753 in PRINCIPAL_DIAGNOSIS_CODE is inferred from the number of elements in the vocabulary_file ./vocab/PRINCIPAL_DIAGNOSIS_CODE_vocab.txt.
INFO:tensorflow:vocabulary_size = 6753 in PRINCIPAL_DIAGNOSIS_CODE is inferred from the number of elements in the vocabulary_file ./vocab/PRINCIPAL_DIAGNOSIS_CODE_vocab.txt.


### Create one-hot encoding  from vocab column feature function

In [81]:
# Explanation of Code Block: Creating a One-Hot Encoded Feature for Principal Diagnosis Code
one_hot_principal_diagnosis_feature = tf.feature_column.indicator_column(principal_diagnosis_vocab)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [82]:
# Explanation of Code Block: Displaying the Created TensorFlow Categorical Feature
categorical_tf_ds_batch = next(iter(categorical_tf_ds))[0]

In [83]:
# Explanation of Code Block: Demonstrating the One-Hot Encoded Feature for Principal Diagnosis Code
demo(one_hot_principal_diagnosis_feature, categorical_tf_ds_batch)

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'